In [ ]:
### Import libraries
from netCDF4 import Dataset
import xarray as xr
import numpy as np
import cartopy.crs as crs
import matplotlib
from matplotlib.cm import get_cmap
import matplotlib.pyplot as plt
from wrf import getvar, interplevel, smooth2d
import warnings
from scipy.interpolate import griddata
from matplotlib.cm import get_cmap
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
warnings.filterwarnings('ignore')
import matplotlib.transforms as mtransforms
import matplotlib.ticker as mticker


In [ ]:
### Read in the data

zl_ctl = np.load('/p/work1/lloveras/nov2018/proc_oct/4km_files/ctl/z400_11-15_12.npy')
zl_adj = np.load('/p/work1/lloveras/nov2018/proc_oct/4km_files/adj_full/z400_11-15_12.npy')
zl_anl = np.load('/p/work1/lloveras/nov2018/proc_oct/4km_files/gfs_anl/z400_11-15_12.npy')

pvl_ctl = np.load('/p/work1/lloveras/nov2018/proc_oct/4km_files/ctl/pv400_11-15_12.npy')
pvl_adj = np.load('/p/work1/lloveras/nov2018/proc_oct/4km_files/adj_full/pv400_11-15_12.npy')
pvl_anl = np.load('/p/work1/lloveras/nov2018/proc_oct/4km_files/gfs_anl/pv400_11-15_12.npy')

pvlay_ctl = np.load('/p/work1/lloveras/nov2018/proc_oct/4km_files/ctl/pv300-500_11-15_12.npy')
pvlay_adj = np.load('/p/work1/lloveras/nov2018/proc_oct/4km_files/adj_full/pv300-500_11-15_12.npy')
pvlay_anl = np.load('/p/work1/lloveras/nov2018/proc_oct/4km_files/gfs_anl/pv300-500_11-15_12.npy') 

lats = np.load('/p/work1/lloveras/nov2018/proc_oct/4km_files/lats.npy')
lons = np.load('/p/work1/lloveras/nov2018/proc_oct/4km_files/lons.npy')


In [ ]:
### Response function box

coamps_dir = '/p/work1/lloveras/nov2018/coamps_files/'

nx_c = 301
ny_c = 201
nz_c = 45

rf = np.fromfile(coamps_dir + 'rfbox1_sig_028485_000010_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4')
rf = np.roll(np.flip(np.reshape(rf, [nz_c, ny_c, nx_c]),axis=0),-4,axis=-1)

lats_c = np.fromfile(coamps_dir + 'latitu_sfc_000000_000000_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4')
lons_c = np.fromfile(coamps_dir + 'longit_sfc_000000_000000_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4')
lats_c = np.reshape(lats_c, [ny_c, nx_c])
lons_c = np.reshape(lons_c, [ny_c, nx_c])
lons_c[lons_c > 0] -= 361


In [ ]:
### Plot subdomain
min_lat = 24
max_lat = 45
min_lon = -105
max_lon = -75

### Contour intervals
zlvls = np.arange(0,1500,10)
dzlvls = np.asarray([-9,-7,-5,-3,-1,1,3,5,7,9])*0.75

### Cartopy projection
proj = crs.LambertConformal(central_longitude=-101,standard_parallels=(20,70))

fig, axd = plt.subplot_mosaic([['left','right'],['cbar','cbar']],
                              constrained_layout=True, figsize=(6.9,3.6), dpi=200, 
                              gridspec_kw={'width_ratios':[1,1],'height_ratios':[1,0.05]},
                              per_subplot_kw={'left':{'projection':proj},
                                              'right':{'projection':proj}})

trans = mtransforms.ScaledTranslation(1/30, -1/30, fig.dpi_scale_trans)

##############
### LEFT
#############

# Add the gridlines
gls = axd['left'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
gls.top_labels=False
gls.right_labels=False
gls.rotate_labels = False

axd['left'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())

cs1 = axd['left'].contour(lons, lats, smooth2d(zl_anl,10)/10.,levels=zlvls,
                colors='k',transform=crs.PlateCarree())
axd['left'].clabel(cs1,fmt='%1.0f',inline=1,levels=zlvls,fontsize=6,colors='k')

cs2 = axd['left'].contour(lons, lats, smooth2d(zl_ctl,10)/10.,levels=zlvls,
                colors='magenta',transform=crs.PlateCarree())
axd['left'].clabel(cs2,fmt='%1.0f',inline=1,levels=zlvls,fontsize=6,colors='magenta')

im1 = axd['left'].contourf(lons, lats, pvlay_ctl - pvlay_anl, levels=dzlvls,
             cmap=get_cmap('RdBu_r'), extend='both',zorder=-20,
             transform=crs.PlateCarree())
axd['left'].set_rasterization_zorder(-19)

axd['left'].contour(lons_c, lats_c, rf[25], levels=[0,1],colors='g',transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['left'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['left'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['left'].set_title('4-km Control')
axd['left'].text(0.0, 1.0, '(a)',transform=axd['left'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### RIGHT
#############

# Add the gridlines
gls = axd['right'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
gls.top_labels=False
gls.right_labels=False
gls.left_labels=False
gls.rotate_labels = False
axd['right'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())

cs1 = axd['right'].contour(lons, lats, smooth2d(zl_anl,10)/10.,levels=zlvls,
                colors='k',transform=crs.PlateCarree())
axd['right'].clabel(cs1,fmt='%1.0f',inline=1,levels=zlvls,fontsize=6,colors='k')

cs2 = axd['right'].contour(lons, lats, smooth2d(zl_adj,10)/10.,levels=zlvls,
                colors='magenta',transform=crs.PlateCarree())
axd['right'].clabel(cs2,fmt='%1.0f',inline=1,levels=zlvls,fontsize=6,colors='magenta')

im1 = axd['right'].contourf(lons, lats, pvlay_adj - pvlay_anl, levels=dzlvls,
             cmap=get_cmap('RdBu_r'), extend='both',zorder=-20,
             transform=crs.PlateCarree())
axd['right'].set_rasterization_zorder(-19)

axd['right'].contour(lons_c, lats_c, rf[25], levels=[0,1],colors='g',transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['right'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['right'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['right'].set_title('4-km Perturbed')
axd['right'].text(0.0, 1.0, '(b)',transform=axd['right'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

### SET THE COLORBAR AND SHOW
cbar = fig.colorbar(im1, orientation='horizontal', cax=axd['cbar'])
plt.savefig('/p/work1/lloveras/nov2018/figs_png/4km_pv.png',bbox_inches='tight')
plt.show()
